In [1]:
import sys; sys.path.append('..') # add project root to the python path

In [2]:
import torch
import gym
import numpy as np

from src.part3.MLP import MultiLayerPerceptron as MLP
from src.part5.DQN import prepare_training_inputs
from src.part5.DDPG import DDPG, Actor, Critic
from src.part5.DDPG import OrnsteinUhlenbeckProcess as OUProcess
from src.common.memory.memory import ReplayMemory
from src.common.train_utils import to_tensor
from src.common.target_update import soft_update

In [3]:
FROM_SCRATCH = True
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
#DEVICE = 'cpu'

## Deep deterministic Policy Gradient (DDPG)

```python
class DDPG(nn.Module):

    def __init__(self,
                 critic: nn.Module,
                 critic_target: nn.Module,
                 actor: nn.Module,
                 actor_target: nn.Module,
                 lr_critic: float = 0.0005,
                 lr_actor: float = 0.001,
                 gamma: float = 0.99):

        super(DDPG, self).__init__()
        self.critic = critic
        self.actor = actor
        self.lr_critic = lr_critic
        self.lr_actor = lr_actor
        self.gamma = gamma

        # setup optimizers
        self.critic_opt = torch.optim.Adam(params=self.critic.parameters(),
                                           lr=lr_critic)

        self.actor_opt = torch.optim.Adam(params=self.actor.parameters(),
                                          lr=lr_actor)

        # setup target networks
        critic_target.load_state_dict(critic.state_dict())
        self.critic_target = critic_target
        actor_target.load_state_dict(actor.state_dict())
        self.actor_target = actor_target

        self.criteria = nn.SmoothL1Loss()

    def get_action(self, state):
        with torch.no_grad():
            a = self.actor(state)
        return a

    def update(self, state, action, reward, next_state, done):
        s, a, r, ns = state, action, reward, next_state

        # compute critic loss and update the critic parameters
        with torch.no_grad():
            critic_target = r + self.gamma * self.critic_target(ns, self.actor_target(ns)) #* (1 - done)
        critic_loss = self.criteria(self.critic(s, a), critic_target)

        self.critic_opt.zero_grad()
        critic_loss.backward()
        self.critic_opt.step()

        # compute actor loss and update the actor parameters
        actor_loss = -self.critic(s, self.actor(s)).mean()  # !!!! Impressively simple
        self.actor_opt.zero_grad()
        actor_loss.backward()
        self.actor_opt.step()
```

In [4]:
lr_actor = 0.005
lr_critic = 0.001
gamma = 0.99
batch_size  = 256
memory_size = 50000
tau = 0.001 # polyak parameter for soft target update
sampling_only_until = 2000

In [5]:
actor, actor_target = Actor(), Actor()
critic, critic_target = Critic(), Critic()

agent = DDPG(critic=critic,
             critic_target=critic_target,
             actor=actor,
             actor_target=actor_target).to(DEVICE)

memory = ReplayMemory(memory_size)

In [6]:
total_eps = 200
print_every = 10

env = gym.make('Pendulum-v0')

if FROM_SCRATCH:
    for n_epi in range(total_eps):
        ou_noise = OUProcess(mu=np.zeros(1))
        s = env.reset()
        cum_r = 0

        while True:
            s = to_tensor(s, size=(1, 3)).to(DEVICE)
            a = agent.get_action(s).cpu().numpy() + ou_noise()[0]
            ns, r, done, info = env.step(a)

            experience = (s,
                          torch.tensor(a).view(1, 1),
                          torch.tensor(r).view(1, 1),
                          torch.tensor(ns).view(1, 3),
                          torch.tensor(done).view(1, 1))
            memory.push(experience)

            s = ns
            cum_r += r

            if len(memory) >= sampling_only_until:
                # train agent
                sampled_exps = memory.sample(batch_size)
                sampled_exps = prepare_training_inputs(sampled_exps, device=DEVICE)
                agent.update(*sampled_exps)
                # update target networks
                soft_update(agent.actor, agent.actor_target, tau)
                soft_update(agent.critic, agent.critic_target, tau)        

            if done:
                break

        if n_epi % print_every == 0:
            msg = (n_epi, cum_r) # ~ -100 cumulative reward = "solved"
            print("Episode : {} | Cumulative Reward : {} |".format(*msg))

    torch.save(agent.state_dict(), 'ddpg_cartpole_user_trained.ptb')
else:
    agent.load_state_dict(torch.load('ddpg_cartpole.ptb'))

Episode : 0 | Cumulative Reward : [-1079.0072] |
Episode : 10 | Cumulative Reward : [-1017.71313] |
Episode : 20 | Cumulative Reward : [-1071.6826] |
Episode : 30 | Cumulative Reward : [-973.743] |
Episode : 40 | Cumulative Reward : [-1522.6953] |
Episode : 50 | Cumulative Reward : [-1087.3086] |
Episode : 60 | Cumulative Reward : [-1382.2721] |
Episode : 70 | Cumulative Reward : [-1353.9075] |
Episode : 80 | Cumulative Reward : [-127.282394] |
Episode : 90 | Cumulative Reward : [-896.93646] |
Episode : 100 | Cumulative Reward : [-265.30908] |
Episode : 110 | Cumulative Reward : [-885.4285] |
Episode : 120 | Cumulative Reward : [-499.44412] |
Episode : 130 | Cumulative Reward : [-118.449135] |
Episode : 140 | Cumulative Reward : [-133.11069] |
Episode : 150 | Cumulative Reward : [-261.5521] |
Episode : 160 | Cumulative Reward : [-252.92813] |
Episode : 170 | Cumulative Reward : [-241.50914] |
Episode : 180 | Cumulative Reward : [-126.9294] |
Episode : 190 | Cumulative Reward : [-1.2625

In [7]:
env = gym.make('Pendulum-v0')

s = env.reset()
cum_r = 0

while True:
    s = to_tensor(s, size=(1, 3))
    a = agent.get_action(s).numpy()
    ns, r, done, info = env.step(a)
    s = ns
    env.render()
    if done:
        break
    
env.close()

RuntimeError: Expected object of device type cuda but got device type cpu for argument #2 'mat1' in call to _th_addmm